# COVID-19 cases among healthcare workers

#### Updated 4/11/2020

### Load python tools

In [1]:
import pandas as pd
from shapely.geometry import shape
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import openpyxl
import altair as alt
from altair_saver import save
import lxml
import requests
from shapely.geometry import Point, LineString
import geojson
import json
import glob
import io
import os
import pyarrow
from shapely.geometry import Point, LineString, MultiLineString
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Read healthcare worker infections data from CHHS

In [2]:
hcworkers = pd.read_excel('input/HCW tables for LAT_20200408.xlsx')

In [3]:
hcworkers['County'] = hcworkers['County'].str.title()

In [4]:
hcworkers.columns = hcworkers.columns.str.lower()

In [5]:
hcworkers['healthcareworker'] = hcworkers['healthcareworker']\
    .astype(str).apply(lambda x: x.replace('<',''))

In [6]:
hcworkers['healthcareworker'] = hcworkers['healthcareworker'].astype(float)

### Read data from hospital utilization report

In [7]:
pop = pd.read_csv('/Users/mhustiles/data/github/notebooks/coronavirus/output/icus_county_pop_slim.csv')

In [8]:
pop = pop.drop(['id'], axis=1)

In [9]:
pop.head()

,county,total_beds,icu_beds,population,icus_per1000
0,Alameda,3746.0,264.0,1643700.0,16.061325
1,Amador,52.0,6.0,37829.0,15.860847
2,Butte,653.0,59.0,227075.0,25.982605
3,Calaveras,25.0,8.0,45235.0,17.685421
4,Colusa,48.0,6.0,21464.0,27.953783


---

### Read data from *LA Times* coronavirus tracker

In [10]:
counties = pd.read_json('/Users/mhustiles/data/github/coronavirus-tracker/_data/counties/totals/all.json')

In [11]:
testing = pd.read_json('/Users/mhustiles/data/github/coronavirus-tracker/_data/testing/timeseries.json')

In [12]:
counties['fips'] = counties['fips'].astype(str).str.zfill(3)

In [13]:
counties.head()

,fips,county,confirmed_cases,deaths
0,001,Alameda,768,20
1,003,Alpine,2,0
2,005,Amador,7,0
3,007,Butte,14,0
4,009,Calaveras,8,0


---

### CA county geography

In [14]:
county_geo = gpd.read_file('/Users/mhustiles/data/data/gis/ca-counties.geojson')

In [15]:
county_geo.columns = county_geo.columns.str.lower()

In [16]:
len(county_geo)

58

---

### Merge with other dataframes

In [17]:
county_geo_pop = county_geo.merge(pop, left_on='name', right_on='county', how='left')

In [18]:
county_geo_pop_covid = county_geo_pop.merge(counties, on='county', how='left')

In [19]:
county_geo_pop_covid_all = county_geo_pop_covid.merge(hcworkers, on='county', how='left')

---

In [20]:
workers = pd.DataFrame(county_geo_pop_covid_all[['fips_x', 'name', 'total_beds', \
                                    'population', 'confirmed_cases', \
                                    'healthcareworker', 'deaths']])

In [21]:
workers.rename(columns={"fips_x": "fips"}, inplace=True)

In [22]:
workers['hc_cases_share'] = ((workers.healthcareworker / workers.confirmed_cases)*100).round(2)
workers['hc_per_100k_pop'] = ((workers.healthcareworker * 100000)/workers.population).round(2)
workers['hc_per_100_hosp_beds'] = ((workers.healthcareworker * 100)/workers.total_beds).round(2)

In [23]:
workersout = workers[workers['healthcareworker'] > 5].sort_values(by='hc_per_100_hosp_beds', ascending=False)

In [24]:
workersout.head(20)

,fips,name,total_beds,population,confirmed_cases,healthcareworker,deaths,hc_cases_share,hc_per_100k_pop,hc_per_100_hosp_beds
53,06087,Santa Cruz,372.0,273765.0,80.0,13.0,1.0,16.25,4.75,3.49
35,06057,Nevada,176.0,99092.0,34.0,6.0,1.0,17.65,6.05,3.41
3,06083,Santa Barbara,1089.0,443738.0,260.0,37.0,2.0,14.23,8.34,3.40
55,06023,Humboldt,299.0,135768.0,50.0,10.0,0.0,20.00,7.37,3.34
20,06073,San Diego,7361.0,3302833.0,1693.0,246.0,44.0,14.53,7.45,3.34
6,06061,Placer,748.0,380077.0,123.0,25.0,4.0,20.33,6.58,3.34
36,06077,San Joaquin,1243.0,732212.0,257.0,41.0,14.0,15.95,5.60,3.30
41,06013,Contra Costa,1803.0,1133247.0,511.0,56.0,9.0,10.96,4.94,3.11
12,06085,Santa Clara,3740.0,1922200.0,1484.0,109.0,50.0,7.35,5.67,2.91
34,06081,San Mateo,1720.0,765935.0,638.0,46.0,21.0,7.21,6.01,2.67


### Statewide share of confirmed COVID cases that are healthcare workers

In [25]:
'{:,.2f}%'.format((workersout.healthcareworker.sum() / workersout.confirmed_cases.sum()*100))

'8.63%'

In [26]:
workersout.to_csv('output/healthcareworkers.csv')